In [481]:
from bs4 import BeautifulSoup
from bs4 import Comment
import urllib.request

In [482]:
class FoodItem:
    def __init__(self, name, price, description, menuGroup):
        self.name = name
        self.price = price
        self.description = description
        self.menuGroup = menuGroup

    def debugPrint(self):
        print(self.name,' \tCOST ',self.price, '\n',self.description,'\n', self.menuGroup)
        print(' ---- ')

In [483]:
def loadInitialMenu(restaurantName):
    restaurantName = '+'.join(restaurantName.split())
    searchQuery = restaurantName + "+" + "menu"
    url = "https://www.bing.com/search?q=" + searchQuery
    response = urllib.request.urlopen(url)
    return response.read()

In [485]:
def parseInitialMenu(html):
    soup = BeautifulSoup(html, 'html.parser')
    # data we need is all in div with class "b_ans b_top b_topborder"
    menuHtml = soup.find("li", class_="b_ans b_top b_topborder")

    if menuHtml is None:
        exit()
    return menuHtml.find("div", class_="tab-container")

def parseMenuGroups(menuHtml):
    menuHeaders = menuHtml.find("div", class_="tab-head")
    menuHeaders = menuHeaders.find("div", class_="tab-menu tab-hasnav")
    menuHeaders = menuHeaders.find_all('li')
    return [item.text for item in menuHeaders]

menu = {}
def parseMenuItems(menuHtml,menuHeaders,isInitial):

    if isInitial:
        startIndex = 0
        menuContent  = menuHtml.find("div", class_="tab-content")
        menuContent = menuContent.select('div[id^="tab_"]')
    else:
        startIndex = 6
        menuContent = menuHtml.select('div[id^="rcontent_"]')


    for i in range(startIndex,len(menuContent)):
    # for i in range(0,3):
        groupedItems = menuContent[i]
        groupedItemsHtml = groupedItems.find_all('div',class_='tdif')
        menuGroup = menuHeaders[i]
        for item in groupedItemsHtml:
            (name,price) = getItemNamePrice(item)
            description = getItemDesc(item)

            if name not in menu:
                # notice how menuGroup parameter is a list
                foodItem = FoodItem(name,price,description,[menuGroup])
                menu[name] = foodItem
            else:
                foodItem = menu[name]
                if menuGroup not in foodItem.menuGroup:
                    foodItem.menuGroup.append(menuGroup)

In [486]:
def getRestaurantId(html):
    soup = BeautifulSoup(html, 'html.parser')
    comments=soup.find_all(string=lambda text:isinstance(text,Comment))
    for c in comments:
        print(c)
        print("===========")

    return 'YN925x203516883'

def getTabCount(menuHeaders):
    return len(menuHeaders)

def loadAdditionalMenu(html):
    id = getRestaurantId(html)
    tabCount = getTabCount(html)
    url = 'https://www.bing.com/local/menu?tabCount=' + str(tabCount) + '&tabStart=6&othersTab=0&ypid='+ id
    response = urllib.request.urlopen(url)
    return response.read()

def parseAdditionalMenu(html,menuHeaders):
    additionalHtml = loadAdditionalMenu(html)
    soup = BeautifulSoup(additionalHtml, 'html.parser')
    parseMenuItems(soup,menuHeaders,isInitial=False)

In [488]:
def getItemNamePrice(item):
    itemName = ''
    itemPrice = ''
    h4 = item.find('h4')
    if h4:
        if len(h4.contents) > 0:
            itemName = h4.contents[0]
        if len(h4.contents) > 1:
            itemPrice = h4.contents[1].contents[0]
    return itemName, itemPrice

def getItemDesc(item):
    itemDescription = ''
    span = item.find('span',class_='b_demoteText')

    if span and len(span.contents) > 0:
        itemDescription = span.contents[0]
    return itemDescription

In [489]:
html = loadInitialMenu('palomino')

In [490]:
menuHtml = parseInitialMenu(html)
menuHeaders = parseMenuGroups(menuHtml)
parseMenuItems(menuHtml, menuHeaders, isInitial=True)

In [457]:
# parseAdditionalMenu(html,menuHeaders)

IndexError: list index out of range

In [491]:
for item in menu:
    print(menu[item].debugPrint())

The Chop Chop  	COST   
 Rotisserie turkey, wine salami, provolone, balsamic vinaigrette 
 ['Dinner Soup & Salads']
 ---- 
None
Rainbow Carrots - Chimichurri, Aged Balsamic, Creme Fraiche  	COST  $7.00 
  
 ['Dinner Vegetables']
 ---- 
None
Seafood Louie  	COST  $23.00 
 Crab, poached prawns, avocado, asparagus 
 ['Dinner Soup & Salads']
 ---- 
None
Portabella Mushroom Fries  	COST  $6.00 
 Dijon aioli, truffle salt 
 ['Dinner Shared Plates']
 ---- 
None
Tomato-Basil Bruschetta  	COST  $9.00 
 Herb oil, roma tomatoes, pesto 
 ['Dinner Breads']
 ---- 
None
Citrus Salad  	COST  $9.00 
 Belgian endive, grapefruit & oranges, bacon, avocado cream, aged red wine vinaigrette 
 ['Dinner Soup & Salads']
 ---- 
None
Grilled Mushroom Salad  	COST  $7.95 
 Food Specials 
 ['Menu Favorites']
 ---- 
None
Organic Field Greens  	COST  $7.50 
 Cypress grove chevre, pumpkin seeds, evoo 
 ['Dinner Soup & Salads']
 ---- 
None
Maple Marinated Chicken Salad  	COST  $20.00 
 Pecan-crusted blue cheese, pears,